## Import packages

In [17]:
from qiskit_machine_learning.utils import algorithm_globals

algorithm_globals.random_seed = 12345

## Prepare Data

In [18]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
data = df.to_numpy()
X = data[:, :-1]  # Use first 8 features to match Pima structure
y = data[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Scale to match Pima-like ranges
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


## Use PCA to reduce input dimensions

In [19]:
from sklearn.decomposition import PCA

pca = PCA()

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

## Run classical SVM for comparison

In [20]:
n_features = 4

print("Running Classical SVM...")

# Train classical SVM
classical_svm = SVC(kernel='rbf', C=1.0, random_state=1)
classical_svm.fit(X_train_pca[:,:n_features], y_train)

y_pred_train_classical = classical_svm.predict(X_train_pca[:, :n_features])
accuracy_train_classical = accuracy_score(y_train, y_pred_train_classical)
# Predict and evaluate
y_pred_classical = classical_svm.predict(X_test_pca[:,:n_features])
accuracy_classical = accuracy_score(y_test, y_pred_classical)

print(f"Classical SVM Accuracy on Training Data: {accuracy_train_classical: .3f}")
print(f"Classical SVM Accuracy on Test Data: {accuracy_classical:.3f}")


Running Classical SVM...
Classical SVM Accuracy on Training Data:  0.769
Classical SVM Accuracy on Test Data: 0.734


## Set up IBM Quantum

In [21]:
print("\nSetting up quantum backend...")

token = "VMgmA9Gjyklr5uAIq8S6FYXrmKvoVN3QLo714I6KjRBA"
instance_id = "QML"

service = QiskitRuntimeService(channel="ibm_cloud", token=token, instance=instance_id)

# Get available backends
backends = service.backends(simulator=False, operational=True)
print("Available quantum backends:")
for backend in backends:
    print(f"  - {backend.name}: {backend.num_qubits} qubits")

# Select least busy real quantum computer
if backends:
    backend = service.least_busy(operational=True, simulator=False)
    print(f"\nSelected backend: {backend.name}")
else:
    print("No quantum backends available, using simulator")
    backend = service.backend('ibmq_qasm_simulator')
    


Setting up quantum backend...
Available quantum backends:
  - ibm_brisbane: 127 qubits
  - ibm_torino: 133 qubits

Selected backend: ibm_brisbane


## Create Quantum Kernel

In [24]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

fm = ZZFeatureMap(feature_dimension=n_features, entanglement="linear")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=fm)

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel=kernel.evaluate)

svc.fit(X_train_pca[:,:n_features], y_train)



TypeError: ClassifierMixin.score() missing 1 required positional argument: 'y'

In [26]:
score_callable_function = svc.score(X_test_pca[:,:n_features], y_test)

print(f"Callable kernel classification test score: {score_callable_function}")

Callable kernel classification test score: 0.625


In [27]:
score_train = svc.score(X_train_pca[:,:n_features], y_train)

print(f"Callable kernel classification test score: {score_callable_function}")

Callable kernel classification test score: 0.625


In [28]:
score_train

0.7447916666666666

In [29]:
from qiskit_ibm_runtime import SamplerV2

sampler = SamplerV2(backend)

In [30]:
fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=fm)

In [31]:
from sklearn.svm import SVC

svc_ibm = SVC(kernel=kernel.evaluate)

svc_ibm.fit(X_train_pca[:,:n_features], y_train)


IBMInputValueError: 'The instruction ZZFeatureMap on qubits (0, 1, 2, 3) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'

## Run classical SVM for comparison

In [8]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Apply level 3 optimization to our overlap circuit
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
overlap_ibm = pm.run(overlap_circ)

NameError: name 'overlap_circ' is not defined